In [180]:
# importando libs
import pandas as pd
from sqlalchemy import create_engine, text
import psycopg2
import boto3
from io import StringIO
import os
from botocore.exceptions import ClientError
import datetime

In [181]:
# postgreSQL
pg_config = {
  "host": "bd-relacional.cjeco8miq291.us-east-1.rds.amazonaws.com",
  "database":"postgres",
  "user": os.getenv("POSTGRES_USER"),
  "password": os.getenv("POSTGRES_PASS"),
  "port":5432
}


# AWS S3
bucket_name = "323317787666"
s3_prefix = "Tech_Challenge/Data"

In [182]:
# conexão com o postgress
conn = psycopg2.connect(**pg_config)

In [183]:
# conexão com S3
s3 = boto3.client('s3')
region = s3.meta.region_name or "us-east-1"

In [184]:
# verifica se já existe o bucket criado, se não, realiza a criação
try:
  s3.head_bucket(Bucket=bucket_name)
  print(f"Bucket {bucket_name} já existe")
except ClientError as e:
  error_code = int(e.response['Error']['Code'])
  if error_code == 404:
    print(f"Bucket {bucket_name} não existe, criando...")
    if region == "us-east-1":
      s3.create_bucket(Bucket=bucket_name)
    else:
      s3.create_bucket(
        Bucket=bucket_name,
        CreateBucketConfiguration={'LocationConstraint': region}
      )
    print(f"Bucket {bucket_name} criado com sucesso")
  elif error_code == 400:
    print(f"❌ ERRO - Reveja as credenciais AWS")
    raise
  else:
    raise

Bucket 323317787666 já existe


In [185]:
# leitura do df
df = pd.read_excel("https://github.com/gteless/FIAP/raw/refs/heads/main/Fase%203/Tech_Challenge/Data/pnad_covid19_202011_escola_emprestimos_itens_de_limpeza_BR_GR_UF.xlsx")

In [204]:
RAW_PREFIX = f'{s3_prefix}/raw/'
SILVER_PREFIX = f's3://{bucket_name}/{s3_prefix}/silver/' 
GOLD_PREFIX = f's3://{bucket_name}/{s3_prefix}/gold' 

In [187]:
# salvar como csv em memória
csv_buffer = StringIO()
df.to_csv(csv_buffer, index=False)

# Enviar para S3
s3_key = f"{RAW_PREFIX}Consolidado_Raw.csv"
s3.put_object(Bucket=bucket_name, Key=s3_key, Body=csv_buffer.getvalue())
print(f"Arquivo salvo no S3 em: s3://{bucket_name}/{s3_key}")

# fecha conexão 
conn.close


Arquivo salvo no S3 em: s3://323317787666/Tech_Challenge/Data/raw/Consolidado_Raw.csv


<function connection.close>

In [188]:
# Camada Silver
consolidado = pd.read_csv(f's3://{bucket_name}/{RAW_PREFIX}Consolidado_Raw.csv')

In [189]:
consolidado.shape

(14262, 16)

In [190]:
consolidado.describe()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
count,14261,14257,14257,14257,14257,14257,14257,5137,5138,5137,5137,5138,5137,5138,14257,14258
unique,41,3,7,11,29,11,41,4504,4496,5,4495,6,4499,6,12362,6
top,Pessoas que frequentam escola (mil pessoas),Grande Região,Brasil,Rendimento domiciliar per capita,Total,Medida tomada de restrição de contato,Não fez restrição,100,0,-,0,-,0,-,0,-
freq,804,11880,2376,4680,2160,3168,792,120,122,4920,121,4920,122,4920,438,14040


In [191]:
consolidado.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,PNAD Covid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Periodicidade da divulgação: Mensal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Nota:os indicadores de mercado de trabalho são...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Nota 1: Inclusive pessoas de cor amarela ou in...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Indicador,Nível Territorial,Abertura Territorial,Variável de abertura 1,Categoria de abertura 1,Variável de abertura 2,Categoria de abertura 2,Julho,Agosto,Situação Julho para Agosto,Setembro,Situação Agosto para Setembro,Outubro,Situação Setembro para Outubro,Novembro,Situação Outubro para Novembro
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Maio,NaN,NaN,Maio,NaN,Maio,NaN,Maio
6,Pessoas que frequentam escola (mil pessoas),País,Brasil,Sexo,Total,-,-,45257.17492492197,45759.70663584802,Aumento,46143.03085570921,Aumento,46412.489523635995,Aumento,46280.85984037859,Estável
7,Pessoas que frequentam escola (mil pessoas),País,Brasil,Sexo,Homem,-,-,22658.6271019043,22972.720473584763,-,23117.787990422366,-,23194.494807600757,-,23143.015789290243,-
8,Pessoas que frequentam escola (mil pessoas),País,Brasil,Sexo,Mulher,-,-,22598.54782301768,22786.98616226326,-,23025.24286528687,-,23217.994716035246,-,23137.844051088316,-
9,Pessoas que frequentam escola (mil pessoas),Grande Região,Norte,Sexo,Total,-,-,4773.232956930837,4836.37704630858,Estável,4878.531954698591,Estável,4914.278376161013,Estável,4883.653085899478,Estável


In [192]:
consolidado.columns = consolidado.iloc[4]
consolidado = consolidado.drop(index=range(0, 5))
consolidado = consolidado.reset_index(drop=True)

In [193]:
consolidado.head()

4,Indicador,Nível Territorial,Abertura Territorial,Variável de abertura 1,Categoria de abertura 1,Variável de abertura 2,Categoria de abertura 2,Julho,Agosto,Situação Julho para Agosto,Setembro,Situação Agosto para Setembro,Outubro,Situação Setembro para Outubro,Novembro,Situação Outubro para Novembro
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Maio,NaN,NaN,Maio,NaN,Maio,NaN,Maio
1,Pessoas que frequentam escola (mil pessoas),País,Brasil,Sexo,Total,-,-,45257.17492492197,45759.70663584802,Aumento,46143.03085570921,Aumento,46412.489523635995,Aumento,46280.85984037859,Estável
2,Pessoas que frequentam escola (mil pessoas),País,Brasil,Sexo,Homem,-,-,22658.6271019043,22972.720473584763,-,23117.787990422366,-,23194.494807600757,-,23143.015789290243,-
3,Pessoas que frequentam escola (mil pessoas),País,Brasil,Sexo,Mulher,-,-,22598.54782301768,22786.98616226326,-,23025.24286528687,-,23217.994716035246,-,23137.844051088316,-
4,Pessoas que frequentam escola (mil pessoas),Grande Região,Norte,Sexo,Total,-,-,4773.232956930837,4836.37704630858,Estável,4878.531954698591,Estável,4914.278376161013,Estável,4883.653085899478,Estável


In [194]:
# padronizar colunas para lower case
consolidado.columns = [col.lower() for col in consolidado.columns]
consolidado.columns = [col.replace(" ", "_") for col in consolidado.columns]

In [195]:
consolidado.head()

,indicador,nível_territorial,abertura_territorial,variável_de_abertura_1,categoria_de_abertura_1,variável_de_abertura_2,categoria_de_abertura_2,julho,agosto,situação_julho_para_agosto,setembro,situação_agosto_para_setembro,outubro,situação_setembro_para_outubro,novembro,situação_outubro_para_novembro
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Maio,NaN,NaN,Maio,NaN,Maio,NaN,Maio
1,Pessoas que frequentam escola (mil pessoas),País,Brasil,Sexo,Total,-,-,45257.17492492197,45759.70663584802,Aumento,46143.03085570921,Aumento,46412.489523635995,Aumento,46280.85984037859,Estável
2,Pessoas que frequentam escola (mil pessoas),País,Brasil,Sexo,Homem,-,-,22658.6271019043,22972.720473584763,-,23117.787990422366,-,23194.494807600757,-,23143.015789290243,-
3,Pessoas que frequentam escola (mil pessoas),País,Brasil,Sexo,Mulher,-,-,22598.54782301768,22786.98616226326,-,23025.24286528687,-,23217.994716035246,-,23137.844051088316,-
4,Pessoas que frequentam escola (mil pessoas),Grande Região,Norte,Sexo,Total,-,-,4773.232956930837,4836.37704630858,Estável,4878.531954698591,Estável,4914.278376161013,Estável,4883.653085899478,Estável


In [196]:
# limpeza básica
consolidado = consolidado.dropna()

In [197]:
consolidado.head()

,indicador,nível_territorial,abertura_territorial,variável_de_abertura_1,categoria_de_abertura_1,variável_de_abertura_2,categoria_de_abertura_2,julho,agosto,situação_julho_para_agosto,setembro,situação_agosto_para_setembro,outubro,situação_setembro_para_outubro,novembro,situação_outubro_para_novembro
1,Pessoas que frequentam escola (mil pessoas),País,Brasil,Sexo,Total,-,-,45257.17492492197,45759.70663584802,Aumento,46143.03085570921,Aumento,46412.489523635995,Aumento,46280.85984037859,Estável
2,Pessoas que frequentam escola (mil pessoas),País,Brasil,Sexo,Homem,-,-,22658.6271019043,22972.720473584763,-,23117.787990422366,-,23194.494807600757,-,23143.015789290243,-
3,Pessoas que frequentam escola (mil pessoas),País,Brasil,Sexo,Mulher,-,-,22598.54782301768,22786.98616226326,-,23025.24286528687,-,23217.994716035246,-,23137.844051088316,-
4,Pessoas que frequentam escola (mil pessoas),Grande Região,Norte,Sexo,Total,-,-,4773.232956930837,4836.37704630858,Estável,4878.531954698591,Estável,4914.278376161013,Estável,4883.653085899478,Estável
5,Pessoas que frequentam escola (mil pessoas),Grande Região,Norte,Sexo,Homem,-,-,2380.020363386206,2425.4535610754006,-,2438.4879929362637,-,2462.5553066560547,-,2439.2895797101455,-


In [198]:
consolidado.shape

(5136, 16)

In [205]:
# salvar camada silver em parquet no S3
consolidado.to_parquet(SILVER_PREFIX + 'consolidado/', index=False)